In [7]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_log_error
from sklearn.ensemble import RandomForestRegressor
from lightgbm import LGBMRegressor
from sklearn import base

import warnings
warnings.filterwarnings('ignore')



In [8]:
from __future__ import division
import pandas as pd
import numpy as np
import datetime
import re
import time
import matplotlib.pyplot as plt
pd.set_option('display.max_columns', 200)
pd.set_option('display.max_rows', 500)
from sklearn.model_selection import train_test_split

from sklearn.preprocessing import LabelEncoder, MinMaxScaler
import xgboost as xgb
from math import sqrt
from sklearn.metrics import mean_squared_error
from scipy.stats import uniform, randint
from sklearn.model_selection import TimeSeriesSplit, cross_val_score, GridSearchCV, RandomizedSearchCV
from sklearn.preprocessing import StandardScaler
#import pydot_ng as pydot
from IPython.display import Image
import math
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
from keras.models import Sequential, Model
from keras.layers import Input, Dense, Activation, Reshape
from keras.layers import Concatenate, Dropout
from keras.layers.embeddings import Embedding
from keras.utils import plot_model
from keras.optimizers import SGD
from sklearn.model_selection import KFold, cross_val_score

In [426]:
#https://www.analyticsvidhya.com/blog/2018/09/multivariate-time-series-guide-forecasting-modeling-python-codes/

In [10]:
training_data = pd.read_csv("../data/TrainingSet.csv",index_col=0)
submission_labels = pd.read_csv("../data/SubmissionRows.csv", index_col=0)

In [ ]:
#df = training_data.melt(id_vars=['Series Code', 'Country Name','Series Name'], var_name='Year', value_name='indicator',ignore_index = False)
'''
df = training_data.melt(id_vars=['Series Code', 'Country Name'], var_name='Year', value_name='indicator',ignore_index = False, value_vars=['1972 [YR1972]','1973 [YR1973]','1974 [YR1974]','1975 [YR1975]',\
                                                                                                                                          '1976 [YR1976]','1977 [YR1977]','1978 [YR1978]','1979 [YR1979]', \
                                                                                                                                          '1980 [YR1980]',  '1981 [YR1981]', '1982 [YR1982]', '1983 [YR1983]', \
                                                                                                                                          '1984 [YR1984]','1985 [YR1985]','1986 [YR1986]','1987 [YR1987]', \
                                                                                                                                          '1988 [YR1988]','1989 [YR1989]','1990 [YR1990]','1991 [YR1991]', \
                                                                                                                                          '1992 [YR1992]','1993 [YR1993]','1994 [YR1994]','1995 [YR1995]', \
                                                                                                                                          '1996 [YR1996]', '1997 [YR1997]','1998 [YR1998]','1999 [YR1999]', \
                                                                                                                                          '2000 [YR2000]','2001 [YR2001]','2002 [YR2002]','2003 [YR2003]' , \
                                                                                                                                          '2004 [YR2004]','2005 [YR2005]','2006 [YR2006]','2007 [YR2007]'])


'''
df = training_data.melt(id_vars=['Series Code', 'Country Name'], var_name='Year', value_name='indicator',ignore_index = False, value_vars=['1996 [YR1996]', '1997 [YR1997]','1998 [YR1998]','1999 [YR1999]', \
                                                                                                                                          '2001 [YR2001]','2002 [YR2002]','2003 [YR2003]' , \
                                                                                                                                          '2004 [YR2004]','2005 [YR2005]','2006 [YR2006]','2007 [YR2007]'])

df['Year'] = df['Year'].str.extract('(\d+)', expand=False) #.4stype(i4t),'2005 [YR2005]','2006 [YR2006]'

#df = df.sort_values(['Week', 'Product_Code'])

pd.set_option('display.max_colwidth', -1)

df['Year'] = pd.to_datetime(df['Year'], format='%Y')


#df = df.head(200)


#df['indicator'] = df.groupby(['Series Code','Country Name','Year'], sort=False)['indicator'].apply(lambda x: x.fillna(x.mean()))

df=df[df.indicator.notnull()]

#df['indicator']=df.groupby(['Series Code','Country Name','Year'])['indicator'].transform(lambda x: x.fillna(x.mean(),inplace=True)))



#df["indicator"] = df.groupby(['Series Code','Country Name','Year'])['indicator'].transform(lambda x: x.fillna(x.avg(),inplace=True))

#df.fillna(0, inplace=True)

#df['indicator'] = df.groupby(['Series Code','Country Name','Year'], sort=False)['indicator'].apply(lambda x: x.fillna(x.mean(),inplace=True))


df.head()


In [15]:
# replace 

df.groupby(['Series Code','Country Name','Year']).mean()

indicator
Series Code   Country Name Year                 
1.2           Albania      2001-01-01  0.00000  
                           2002-01-01  0.00090  
                           2003-01-01  0.00000  
                           2004-01-01  0.00110  
                           2005-01-01  0.00170  
...                                        ...  
lm_ub.gen_pop Ukraine      2003-01-01  0.00000  
                           2004-01-01  0.00000  
                           2005-01-01  0.00000  
                           2006-01-01  6.96192  
                           2007-01-01  0.00000  

[1367814 rows x 1 columns]

In [64]:
# target Encoding Does not Work 
'''
target_encode_columns = ['Country Name','Series Code']
target = ['indicator']
target_encode_df = df[target_encode_columns + target] #.reset_index().drop(columns = 'index', axis = 1)
target_name = target[0]
target_df = pd.DataFrame()
for embed_col in target_encode_columns:
    val_map = target_encode_df.groupby(embed_col)[target].mean().to_dict()[target_name]
    target_df[embed_col] = target_encode_df[embed_col].map(val_map).values
    
score_target_drop = df.drop(target_encode_columns, axis = 1).reset_index().drop(columns = 'index', axis = 1)
score_target = pd.concat([score_target_drop, target_df], axis = 1)

score_target['Year']=score_target['Year'].dt.year
score_target.head()
'''


# Label Encoding 

encode_columns = ['Country Name','Series Code']
encode_df = df[encode_columns]
encode_df = encode_df.astype('str')
encode_df = encode_df.apply(LabelEncoder().fit_transform)
score_encode_drop = df.drop(encode_columns, axis = 1)
#score_encode_drop=df

score_encode = pd.concat([score_encode_drop, encode_df], axis = 1)
score_encode['Year']=df['Year'].dt.year
score_target=score_encode
#score_target.groupby('Country Name').nunique()
score_target.head()

,Year,indicator,Country Name,Series Code
29,1972,1.383995e-01,0,752
30,1972,2.257446e+06,0,751
33,1972,1.029650e+00,0,750
34,1972,1.679471e+07,0,749
35,1972,1.369047e+00,0,760


In [54]:
#https://www.kaggle.com/robikscube/tutorial-time-series-forecasting-with-xgboost
#not working

In [67]:
#Another try 

split_date = 2006
df_train = score_target.loc[score_target['Year'] <= split_date].copy()
df_test = score_target.loc[score_target['Year'] > split_date].copy()


def create_features(df, label=None):

    df['Year'] =  pd.to_datetime(df['Year'], format='%Y').dt.year

    X = df[['Country Name','Series Code','Year']]
    
    if label:
        y = df[label]
        return X, y
    return X

X_train, y_train = create_features(df_train, label='indicator')
X_test, y_test = create_features(df_test, label='indicator')


params = {'colsample_bytree': uniform(0.7, 0.3),
          'gamma': uniform(0, 0.5),
          'learning_rate': uniform(0.003, 0.1), # default 0.1 
          'max_depth': randint(2, 6), # default 3
          'n_estimators': randint(100, 250), # default 100
          'subsample': uniform(0.6, 0.4)}
xgb_model = xgb.XGBRegressor(objective="reg:squarederror",random_state=42)
time_split = TimeSeriesSplit(n_splits = 2)
xgb_search = RandomizedSearchCV(xgb_model,param_distributions=params, random_state=42, n_iter=4, cv=time_split, verbose=1, n_jobs=1, return_train_score=True)
xgb_search.fit(X_train,y_train)


Fitting 2 folds for each of 4 candidates, totalling 8 fits


RandomizedSearchCV(cv=TimeSeriesSplit(gap=0, max_train_size=None, n_splits=2, test_size=None),
                   estimator=XGBRegressor(base_score=None, booster=None,
                                          colsample_bylevel=None,
                                          colsample_bynode=None,
                                          colsample_bytree=None, gamma=None,
                                          gpu_id=None, importance_type='gain',
                                          interaction_constraints=None,
                                          learning_rate=None,
                                          max_delta_step=None, max_depth=None,
                                          min_child_w...
                                        'learning_rate': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7fc45deb67d0>,
                                        'max_depth': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7fc45deb6cd0>,
                     

In [68]:
y_pred = xgb_search.predict(X_test)
rms = sqrt(mean_squared_error(y_test, y_pred))
print ('RMSE:', rms)

RMSE: 36728995131704.34


In [70]:

print(y_test[0],y_pred[0])

3.76921388935098 -525089370000.0


In [71]:
#Random forest

model = RandomForestRegressor(n_estimators=1000)
model.fit(X_train, y_train)

MemoryError: could not allocate 234881024 bytes

In [ ]:
'''
# Create features from datetime index
def create_features(df, label=None):
    df['date'] = df.index
    df['hour'] = df['date'].dt.hour
    df['dayofweek'] = df['date'].dt.dayofweek
    df['month'] = df['date'].dt.month
    df['quarter'] = df['date'].dt.quarter
    df['year'] = df['date'].dt.year
    df['dayofyear'] = df['date'].dt.dayofyear
    df['dayofmonth'] = df['date'].dt.day
    df['weekofyear'] = df['date'].dt.weekofyear
    
    X = df[['hour', 'dayofweek','month', 'quarter', 'year', 'dayofyear',
            'dayofmonth', 'weekofyear']]
    if label:
        y = df[label]
        return X, y
    return X

y_train=score_target['indicator']
y_test=score_target['indicator']

reg = xgb.XGBRegressor(n_estimators=1000)
reg.fit(X_train, y_train,
        eval_set=[(X_train, y_train), (X_test, y_test)],
        early_stopping_rounds = 50,
        verbose = False)
'''

In [142]:
'''X=score_target[['Year','Country Name','Series Code']]
y=score_target['indicator']

X_train = X[:int(X.shape[0]*0.7)]
X_test = X[int(X.shape[0]*0.7):]
y_train = y[:int(X.shape[0]*0.7)]
y_test = y[int(X.shape[0]*0.7):]

params = {'colsample_bytree': uniform(0.7, 0.3),
          'gamma': uniform(0, 0.5),
          'learning_rate': uniform(0.003, 0.1), # default 0.1 
          'max_depth': randint(2, 6), # default 3
          'n_estimators': randint(100, 250), # default 100
          'subsample': uniform(0.6, 0.4)}
xgb_model = xgb.XGBRegressor(objective="reg:squarederror",random_state=42)
time_split = TimeSeriesSplit(n_splits = 10)
xgb_search = RandomizedSearchCV(xgb_model,param_distributions=params, random_state=42, n_iter=4, cv=time_split, verbose=1, n_jobs=1, return_train_score=True)
xgb_search.fit(X_train,y_train)
'''


Fitting 10 folds for each of 4 candidates, totalling 40 fits


RandomizedSearchCV(cv=TimeSeriesSplit(gap=0, max_train_size=None, n_splits=10, test_size=None),
                   estimator=XGBRegressor(base_score=None, booster=None,
                                          colsample_bylevel=None,
                                          colsample_bynode=None,
                                          colsample_bytree=None, gamma=None,
                                          gpu_id=None, importance_type='gain',
                                          interaction_constraints=None,
                                          learning_rate=None,
                                          max_delta_step=None, max_depth=None,
                                          min_child_...
                                        'learning_rate': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7ffa0733ef90>,
                                        'max_depth': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7ffa0733e650>,
                     

In [125]:
'''
#X_train, X_test,y_train,y_test = train_test_split(score_target[['Year','Country Name','Series Code']],score_target['indicator'], test_size=0.1, random_state=42
#can't use this simple method 
#https://medium.com/keita-starts-data-science/time-series-split-with-scikit-learn-74f5be38489e


params = {'colsample_bytree': uniform(0.7, 0.3),
          'gamma': uniform(0, 0.5),
          'learning_rate': uniform(0.003, 0.1), # default 0.1 
          'max_depth': randint(2, 6), # default 3
          'n_estimators': randint(100, 250), # default 100
          'subsample': uniform(0.6, 0.4)}
xgb_model = xgb.XGBRegressor(objective="reg:squarederror",random_state=42)
time_split = TimeSeriesSplit(n_splits = 4)
xgb_search = RandomizedSearchCV(xgb_model,param_distributions=params, random_state=42, n_iter=4, cv=time_split, verbose=1, n_jobs=1, return_train_score=True)
x_model=xgb_search.fit(X_train,y_train)
x_model.score(X_train,y_train)
'''

Fitting 4 folds for each of 4 candidates, totalling 16 fits


0.3169088164893201

In [144]:
y_pred = xgb_search.predict(X_test)
rms = sqrt(mean_squared_error(y_test, y_pred))
print ('RMSE:', rms)

RMSE: 28251739236895.812


In [35]:

#RMSE: 18679478358922.07
#RMSE: 17282979473479.717
#RMSE: 28251739236895.812
#RMSE: 31451691812712.293


# connect predictions with outputs
#for i in range(len(y_pred)):
#	print(X_test.loc[i], "pred is: " + str(y_pred[i]))
    

   Year  Country Name  Series Code
0  1972  0             1296       
0  2002  0             1296       
0  1992  0             1296       
0  2004  0             1296       
0  1974  0             1296       
0  2003  0             1296       
0  1980  0             1296       
0  1977  0             1296       
0  1996  0             1296       
0  1997  0             1296       
0  1988  0             1296       
0  2000  0             1296       
0  1982  0             1296       
0  2005  0             1296       
0  1991  0             1296       
0  1999  0             1296       
0  1989  0             1296        pred is: -92444205000.0
   Year  Country Name  Series Code
1  2000  0             1299       
1  1998  0             1299       
1  1973  0             1299       
1  1995  0             1299       
1  1999  0             1299       
1  1988  0             1299       
1  1977  0             1299       
1  1986  0             1299       
1  1975  0             1299    

KeyError: 3

In [109]:
#print(X_test.iloc[559],y_pred[559])

In [110]:
submission_results=submission_labels.join(training_data)

submission_results=submission_results[['Country Name','Series Code']]

submission_results['Year']=2008

submission_results['indicator']=0

#submission_results.head() 

submission_results.groupby('Year').nunique()

,Country Name,Series Code,indicator
Year,,,
2008,206,7,1


In [ ]:
# Target Encoding does not work 

In [112]:
'''target_encode_columns = ['Country Name','Series Code']
target = ['indicator']
target_encode_df = submission_results[target_encode_columns + target] #.reset_index().drop(columns = 'index', axis = 1)
target_name = target[0]
target_df = pd.DataFrame()
for embed_col in target_encode_columns:
    val_map = target_encode_df.groupby(embed_col)[target].mean().to_dict()[target_name]
    target_df[embed_col] = target_encode_df[embed_col].map(val_map).values
    
score_predict_drop = submission_results.drop(target_encode_columns, axis = 1).reset_index().drop(columns = 'index', axis = 1)
score_predict = pd.concat([score_target_drop, target_df], axis = 1)

score_predict['Year']= pd.to_datetime(score_target['Year'], format='%Y').dt.year
'''

# Label Encoding 

encode_columns = ['Country Name','Series Code']
pred_df = submission_results[encode_columns]
score_predict_drop = submission_results.drop(encode_columns, axis = 1)
pred_df = pred_df.astype('str')
pred_df = pred_df.apply(LabelEncoder().fit_transform)


score_predict_2008 = pd.concat([score_predict_drop, pred_df], axis = 1)

score_predict['Year']=pd.to_datetime(score_predict_2008['Year'],format='%Y').dt.year

res_pred_01 = xgb_search.predict(score_predict_2008[['Year', 'Country Name', 'Series Code']])

score_predict_2008['predictions'] = res_pred_01

score_predict_2008.head()



,Year,indicator,Country Name,Series Code,predictions
559,2008,0,0,5,-4.474982e+11
618,2008,0,0,6,-4.474982e+11
753,2008,0,0,1,-4.474982e+11
1030,2008,0,0,3,-4.474982e+11
1896,2008,0,1,5,-4.474982e+11


In [78]:
submission_results=submission_labels.join(training_data)

submission_results=submission_results[['Country Name','Series Code']]

submission_results['Year']=2009

submission_results['indicator']=0

# Label Encoding 

encode_columns = ['Country Name','Series Code']
pred_df = submission_results[encode_columns]
score_predict_drop = submission_results.drop(encode_columns, axis = 1)
pred_df = pred_df.astype('str')
pred_df = pred_df.apply(LabelEncoder().fit_transform)


score_predict_2009 = pd.concat([score_predict_drop, pred_df], axis = 1)

score_predict_2009['Year']=pd.to_datetime(score_predict_2009['Year'],format='%Y').dt.year

res_pred_01 = xgb_search.predict(score_predict_2009[['Year', 'Country Name', 'Series Code']])

score_predict_2009['predictions'] = res_pred_01

score_predict_2009.head()



,Year,indicator,Country Name,Series Code,predictions
559,2009,0,0,5,-8.041746e+11
618,2009,0,0,6,-8.041746e+11
753,2009,0,0,1,-8.041746e+11
1030,2009,0,0,3,-8.041746e+11
1896,2009,0,1,5,-8.041746e+11


In [97]:
predictions_from_model=score_predict_2009.join(score_predict_2008,how='inner',lsuffix=2009, rsuffix=2008)


predictions_from_model=predictions_from_model.drop(columns=['Year2009','indicator2009','Country Name2009','Year2008','indicator2008','Country Name2008','Series Code2009','Series Code2008'])


predictions_from_model=predictions_from_model.rename(columns={"predictions2008": "2008 [YR2008]", "predictions2009": "2009 [YR2009]"})

predictions_from_model.to_csv("../data/result_02.csv")
